There are some POIs that are not in Switzerland. We will delete them from the database by using the following function.

In [35]:
from pymongo import MongoClient

client = MongoClient(host="localhost",port=27017)
db = client["data_base_OSM"]
collection = db["bicycle_amenities"]

In [17]:
wrong_POIs = ["Baden-Württemberg", "Vorarlberg", "Trentino-Alto Adige/Südtirol", "Lombardia", "Auvergne-Rhône-Alpes",
              "No canton found", "Grand Est", "Bourgogne-Franche-Comté", "Valle d'Aosta / Vallée d'Aoste", "Piemonte"]


def delete_wrong_POIs(wrong_POIs):
    collection.delete_many({"node.canton": {"$in": wrong_POIs}})
    print(f"Deleted documents with canton '{wrong_POIs}'")


delete_wrong_POIs(wrong_POIs)

Deleted documents with canton '['Baden-Württemberg', 'Vorarlberg', 'Trentino-Alto Adige/Südtirol', 'Lombardia', 'Auvergne-Rhône-Alpes', 'No canton found', 'Grand Est', 'Bourgogne-Franche-Comté', "Valle d'Aosta / Vallée d'Aoste"]'


In [18]:
def check_wrong_POIs(wrong_POIs):
    for wrong_POI in wrong_POIs:
        count = collection.count_documents({"node.canton": {"$in": wrong_POIs}})
        print(f"Count of documents with canton '{wrong_POI}': {count}")


check_wrong_POIs(wrong_POIs)

Count of documents with canton 'Baden-Württemberg': 0
Count of documents with canton 'Vorarlberg': 0
Count of documents with canton 'Trentino-Alto Adige/Südtirol': 0
Count of documents with canton 'Lombardia': 0
Count of documents with canton 'Auvergne-Rhône-Alpes': 0
Count of documents with canton 'No canton found': 0
Count of documents with canton 'Grand Est': 0
Count of documents with canton 'Bourgogne-Franche-Comté': 0
Count of documents with canton 'Valle d'Aosta / Vallée d'Aoste': 0


The cantons with french or italian names will be translated to german.

In [25]:
canton_translation = {
    "Vaud": "Waadt",
    "Graubünden/Grischun/Grigioni": "Graubünden",
    "Bern/Berne": "Bern",
    "Valais/Wallis": "Wallis",
    "Neuchâtel": "Neuenburg",
    "Ticino": "Tessin",
    "Genève": "Genf",
    "Fribourg/Freiburg": "Freiburg"
}


def translate_canton(canton_translation):
    for key, value in canton_translation.items():
        collection.update_many({"node.canton": key}, {"$set": {"node.canton": value}})
        print(f"Updated documents with canton '{key}' to '{value}'")


translate_canton(canton_translation)

Updated documents with canton 'Vaud' to 'Waadt'
Updated documents with canton 'Graubünden/Grischun/Grigioni' to 'Graubünden'
Updated documents with canton 'Bern/Berne' to 'Bern'
Updated documents with canton 'Valais/Wallis' to 'Wallis'
Updated documents with canton 'Neuchâtel' to 'Neuenburg'
Updated documents with canton 'Ticino' to 'Tessin'
Updated documents with canton 'Genève' to 'Genf'
Updated documents with canton 'Fribourg/Freiburg' to 'Freiburg'


In [6]:
def check_canton_translation(canton_translation):
    for key, value in canton_translation.items():
        count = collection.count_documents({"node.canton": key})
        print(f"Count of documents with canton '{key}': {count}")


check_canton_translation(canton_translation)

Count of documents with canton 'Vaud': 0
Count of documents with canton 'Graubünden/Grischun/Grigioni': 0
Count of documents with canton 'Bern/Berne': 0
Count of documents with canton 'Valais/Wallis': 0
Count of documents with canton 'Neuchâtel': 0
Count of documents with canton 'Ticino': 0
Count of documents with canton 'Genève': 0
Count of documents with canton 'Fribourg/Freiburg': 0


Enrichment of the data with pump stations in the city of Zurich.

In [33]:
import json

data = json.load(open("../data/taz.velopumpstationen_p.json", "r", encoding="utf-8"))

def transform_feature(feature):
    # Extrahiert und transformiert ein einzelnes Feature
    transformed = {
        "amenity": "compressed_air",
        "id": str(feature["properties"]["id1"]),
        "lat": str(feature["geometry"]["coordinates"][1]),
        "lon": str(feature["geometry"]["coordinates"][0]),
        "canton": "Zürich",
        "location": {
            "type": "Point",
            "coordinates": feature["geometry"]["coordinates"]
        }
    }
    return {"node": transformed}

def transform_json(input_data):
    # Transformiert die gesamte FeatureCollection
    transformed_features = [transform_feature(feature) for feature in input_data["features"]]
    return transformed_features

# Transformation durchführen
transformed_json = transform_json(data)

print(transformed_json[0])

json.dump(transformed_json, open("../data/compressed_air_transformed.json", "w", encoding="utf-8"))

{'node': {'amenity': 'compressed_air', 'id': '1011', 'lat': '47.3917066531', 'lon': '8.5185306277', 'canton': 'Zürich', 'location': {'type': 'Point', 'coordinates': [8.5185306277, 47.3917066531]}}}


In [36]:
def load_data_in_db(data):
    collection.insert_many(data)
    print("Inserted documents into database")
    
load_data_in_db(transformed_json)

Inserted documents into database
